In [1]:
# Referensi:

# docx2txt
# https://github.com/ankushshah89/python-docx2txt

# Job Scrape from Web
# https://www.youtube.com/watch?v=eN_3d4JrL_w
# github dari YouTube di atas: https://github.com/israel-dryer/Indeed-Job-Scraper
# https://www.geeksforgeeks.org/how-to-scrape-multiple-pages-of-a-website-using-python/

# Pencocokan Resume dan Job Scrape
# https://randerson112358.medium.com/resume-scanner-2c30f5baf92c

# Our Location
# https://geocoder.readthedocs.io/providers/IPInfo.html#geocode-your-own-ip
# https://stackoverflow.com/questions/30218394/timeout-error-in-python-geopy-geocoder

# Google Translate Library
# https://cloud.google.com/translate/docs/basic/translating-text#translating_text

In [ ]:
# Pencocokkan Resume yang di upload dengan laman pekerjaan Indeed (Indonesia)

import docx2txt
import re
import requests
import geocoder
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from docx import Document
import six
from google.cloud import translate_v2 as translate

# Proses document '.docx' agar bisa dibaca dan menggantikan tab dengan spasi
def find_text_from_docx(docx):
    text = docx2txt.process(docx)
    if text:
        return text.replace('\t', ' ')
    return None


def per_page_scrape(job, place, docx):
    document = Document()
    
    # pattern link per page
    jk_pattern = re.compile(r"jk:\'([a-zA-Z0-9]+)'")
    
    # ganti spasi di input jadi '+' biar nyambung dengan link
    job = job.replace(' ', '+')
    place  = place.replace(' ', '+')
    
    # parameter link
    params = { "q": job, "l": place, "start": 0 }
    url = "https://id.indeed.com/lowongan-kerja"
    job_keys = set()
    
    # contoh link: https://id.indeed.com/lowongan-kerja?q=full+stack&l=jakarta
    
    # Cek per page untuk pattern link
    for page in range(0, 4):
        response = requests.get(url, params=params)
        if not response.status_code == 200:
            break
        else:
            keys = jk_pattern.findall(response.text)
            if len(keys) > 0:
                for key in keys:
                    job_keys.add(key)
            
            template = "https://id.indeed.com/viewjob?jk={}"
            # pattern link di pop ke template
            jk = job_keys.pop()
            # template di format dengan dimasukkan pattern ke {}
            job_url = template.format(jk)
            # akses link
            response = requests.get(job_url)
            
            # Parse Webpage
            soup = BeautifulSoup(response.text, 'html.parser')
            # Buat title di Webpage sebagai penanda di hasil '.docx'
            document.add_paragraph(soup.find("h1", attrs={"class": "icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title"}).text)
            # Cari deskripsi pekerjaan dan dibuat jadi text
            job_description = soup.find("div", id="jobDescriptionText").text
            
            # Translate dari English ke Indonesia
            translate_client = translate.Client()
            if isinstance(job_description, six.binary_type):
                job_description = job_description.decode("utf-8-sig")
            result = translate_client.translate(job_description, target_language="id", model="nmt")
            
            # Hasil translate dibersihkan dengan menghilangkan '&#39;' dan diganti dengan enter
            translated_texts = result["translatedText"]
            clean_job_description = translated_texts.replace('&#39;', '\n')
            
            # Masukkan hasil translate bersih ke '.docx'
            document.add_paragraph(clean_job_description)
            document.add_paragraph(job_url)
            
            # Memasukkan resume dan hasil translate bersih ke dalam 2 masing-masing array
            list = [docx, clean_job_description]
            
            # Penjelasan: https://blog.kmkonline.co.id/were-doing-machine-learning-9d4075d46cc3
            cv = CountVectorizer()
            count_matrix = cv.fit_transform(list)
            
            # Similarity score
            matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
            matchPercentage = round(matchPercentage, 2)
            
            document.add_paragraph("Resume mempunyai kesamaan sebesar: "+ str(matchPercentage)+ "% dari deskripsi pekerjaan.")
            document.add_page_break()
        
        # Untuk pergi ke Webpage selanjutnya
        params['start'] += 10
        sleep(randint(1,4))
        
    # Save document 
    document.save('Hasil.docx')

# Untuk mengambil lokasi kota pengguna
g = geocoder.ipinfo('me')
time.sleep(8)
curr_city = g.city
time.sleep(8)

# Lokasi resume pengguna
file_location = 'Adhi Gopalam - SM.en.id.docx'

# Posisi pekerjaan yang diinginkan
wanted_job_position = "full stack"

if __name__ == '__main__':
    per_page_scrape(wanted_job_position, curr_city, find_text_from_docx(file_location))